In [20]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor
)


from ipymarkup import show_span_box_markup

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
money_extractor = MoneyExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

<ipython-input-3-fd0b9af24538>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [4]:
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

In [39]:
from tqdm import tqdm

In [6]:
with open("avito_cars.txt", encoding="UTF-8") as file:
    text = file.read()
    car_data = text.splitlines()

In [14]:
car_data = pd.Series(car_data, dtype="U")

In [29]:
car_data.shape

(53726,)

In [31]:
Mashina = fact(
    'Mashina',
    ['label', 'model', 'date']
)

In [35]:
years = {
    "96":"1996",
    "97":"1997",
    "98":"1998",
    "99":"1999",
    "2000":"2000",
    "1999":"1999",
    "1998":"1998",
    "1997":"1997",    
    "1996":"1996",
    "11":"2011",
    "12":"2012",
    "13":"2013",
    "14":"2014",
    "15":"2015",
    "16":"2016",
    "17":"2017",
    "18":"2018",
    "2004":"2004",
    "2005":"2005",
    "2006":"2006",
    "2007":"2007",
    "2008":"2008",
    "2009":"2009",
    "2011":"2011",
    "2012":"2012",
    "2013":"2013",
    "2014":"2014",
    "2015":"2015",
    "2016":"2016",
    "2017":"2017",
    "2018":"2018"    
}
vaz_types = {
    "21063":"21063",
    "21074":"21074",
    "2112":"2112",
    "2109":"2109",
    "2106":"2106"
}    
VAZ = rule(
    morph_pipeline(["ВАЗ", "ваз", "Ваз"]).interpretation(Mashina.label.const("ГАЗ")),
    caseless_pipeline(vaz_types).interpretation(Mashina.model.normalized().custom(vaz_types.get)).optional(),
    caseless_pipeline(years).interpretation(Mashina.date.normalized().custom(years.get)).optional()   
)
gaz_types = {
    "330202":"330202",
    "322132":"322132"
}

GAZ = rule(
    morph_pipeline(["ГАЗ", "газ", "Газ"]).interpretation(Mashina.label.const("ГАЗ")),
    caseless_pipeline(gaz_types).interpretation(Mashina.model.normalized().custom(gaz_types.get)).optional(),
    caseless_pipeline(years).interpretation(Mashina.date.normalized().custom(years.get)).optional()
)
uaz_types = {
    "31514":"31514",
    "патриот":"патриот",
    "hunter":"hunter",
    "хантер":"hunter"    
}

UAZ = rule(
    morph_pipeline(["УАЗ", "Уаз", "уаз"]).interpretation(Mashina.label.const("УАЗ")),
    caseless_pipeline(uaz_types).interpretation(Mashina.model.normalized().custom(uaz_types.get)).optional(),
    caseless_pipeline(years).interpretation(Mashina.date.normalized().custom(years.get)).optional()
)
volga_types = {
    "31029":"31029",
    "21":"21",
    "2410":"2410",
    "3110":"3110"    
}

Volga = rule(
    morph_pipeline(["Волга", "волга", "ВОЛГА", "Волгу", "волгу", "ВОЛГУ"]).interpretation(Mashina.label.const("Волга")),
    caseless_pipeline(volga_types).interpretation(Mashina.model.normalized().custom(volga_types.get)).optional(),
    caseless_pipeline(years).interpretation(Mashina.date.normalized().custom(years.get)).optional()
)
niva_types = {
    "шевроле":"шевроле",
    "4х4":"4х4"
}
Niva = rule(
    morph_pipeline(["Нива", "нива", "НИВА", "Ниву", "ниву", "НИВУ"]).interpretation(Mashina.label.const("Нива")),
    caseless_pipeline(niva_types).interpretation(Mashina.model.normalized().custom(niva_types.get)).optional(),
    caseless_pipeline(years).interpretation(Mashina.date.normalized().custom(years.get)).optional()
)
lada_types = {
    "калина":"калина",
    "kalina":"калина",
    "приора":"приора",
    "priora":"приора",
    "гранта":"гранта",
    "granta":"гранта"
}
Lada = rule(
    morph_pipeline(["Lada", "LADA", "lada", "лада", "ЛАДА", "Лада"]).interpretation(Mashina.label.const("Лада")),
    caseless_pipeline(lada_types).interpretation(Mashina.model.normalized().custom(lada_types.get)).optional(),
    caseless_pipeline(years).interpretation(Mashina.date.normalized().custom(years.get)).optional()
)
gazel_types = {
    "278412":"278412",
    "фермер":"фермер",
    "бизнес":"бизнес",
    "3302":"33023",
    "33023":"33023",
    "некст":"некст",
}
Gazel = rule(
    morph_pipeline(["Газель", "ГАЗЕЛЬ", "газель", "ГАЗель"]).interpretation(Mashina.label.const("Газель")),
    caseless_pipeline(gazel_types).interpretation(Mashina.model.normalized().custom(gazel_types.get)).optional(),
    caseless_pipeline(years).interpretation(Mashina.date.normalized().custom(years.get)).optional()
)

In [36]:
MASHINA = or_(VAZ, GAZ, UAZ, Niva, Lada, Volga, Gazel).interpretation(Mashina)

In [37]:
parser = Parser(MASHINA)

In [40]:
matches = []

for sent in tqdm(car_data):
    for match in parser.findall(sent):
        matches.append(match.fact)

100%|███████████████████████████████████████████████████████████████████████████| 53726/53726 [06:35<00:00, 135.87it/s]


In [42]:
def with_opts(item):
    if item.model is not None and item.date is not None:
        return True
    else:
        return False

In [43]:
full_matches = [i for i in filter(with_opts, matches)]

In [44]:
len(full_matches)

220

In [47]:
no_repeat = list(set(full_matches))

In [50]:
for i in no_repeat[:30]:
    print(f"Марка: {i.label},\nмодель: {i.model},\nгод: {i.date}\n")

Марка: Нива,
модель: 4х4,
год: 2006

Марка: Лада,
модель: приора,
год: 2007

Марка: Нива,
модель: 4х4,
год: 2017

Марка: Газель,
модель: бизнес,
год: 2012

Марка: Нива,
модель: 4х4,
год: 1997

Марка: Газель,
модель: фермер,
год: 2004

Марка: Лада,
модель: приора,
год: 2009

Марка: Лада,
модель: гранта,
год: 2015

Марка: УАЗ,
модель: патриот,
год: 2012

Марка: Лада,
модель: калина,
год: 2014

Марка: Лада,
модель: приора,
год: 2015

Марка: Нива,
модель: шевроле,
год: 2012

Марка: ГАЗ,
модель: 2112,
год: 2004

Марка: Лада,
модель: калина,
год: 2005

Марка: Нива,
модель: 4х4,
год: 2014

Марка: ГАЗ,
модель: 2106,
год: 1997

Марка: УАЗ,
модель: патриот,
год: 2013

Марка: Нива,
модель: шевроле,
год: 2006

Марка: Нива,
модель: шевроле,
год: 2013

Марка: Лада,
модель: калина,
год: 2011

Марка: ГАЗ,
модель: 2109,
год: 2000

Марка: УАЗ,
модель: hunter,
год: 2016

Марка: УАЗ,
модель: 31514,
год: 2000

Марка: УАЗ,
модель: hunter,
год: 2006

Марка: УАЗ,
модель: hunter,
год: 2008

Марка: Нива,
модель